In [2]:
import pandas as pd
df=pd.read_csv("train_with_COT.csv")
df.head()

,compound,subset,sentence_type,sentence,expected_order,image1_name,image1_caption,image2_name,image2_caption,image3_name,image3_caption,image4_name,image4_caption,image5_name,image5_caption,COT
0,elbow grease,Train,idiomatic,It took a lot of elbow grease to get the old e...,"['35234427395.png', '53378381715.png', '399382...",35234427395.png,The image depicts a hand holding a sponge and ...,39938261459.png,The image depicts a hand wearing a yellow work...,53378381715.png,The image depicts a hand holding a duster with...,54879908369.png,The image depicts a person wearing knee pads a...,74852536462.png,The image depicts a person wearing a black out...,The correct response based on the step-by-step...
1,night owl,Train,idiomatic,"It's a constant battle for us, as he is a morn...","['61697797701.png', '93189810779.png', '893752...",00982495584.png,The image depicts a nighttime scene with a lar...,61697797701.png,The image depicts a cartoon-style illustration...,89375227504.png,The image depicts a cartoon-style owl perched ...,93189810779.png,The image depicts a cartoon-style illustration...,93541983868.png,"The image depicts a dumbbell, which is a commo...","Let's analyze the phrase ""night owl"" step by s..."
2,heart of gold,Train,idiomatic,Even the somewhat seedy failed private eye has...,"['86137977215.png', '78062290185.png', '542405...",54240592941.png,"The image depicts a large, metallic safe with ...",78062290185.png,The image depicts a joyful scene featuring a y...,86137977215.png,The image depicts a cartoon scene where a youn...,90660547751.png,"The image depicts a futuristic, stylized space...",92088849364.png,"The image depicts a stylized, artistic represe...",Here's the response based on the step-by-step ...
3,agony aunt,Sample,idiomatic,ESA's Space Weather Office is like Europe's st...,"['83600499282.png', '57658144685.png', '025128...",02512838127.png,The image depicts a serene outdoor scene featu...,32964421720.png,The image depicts a cartoon-style illustration...,57658144685.png,The image depicts a scene of a person sitting ...,83600499282.png,"The image depicts a person sitting at a desk, ...",92533456778.png,The image depicts a cartoon character of a wom...,Let's break down the analysis step by step:\n\...
4,shrinking violet,Train,idiomatic,"This aged, rich wine is no shrinking violet wi...","['77861539717.png', '68016869942.png', '118443...",11844321898.png,The image depicts a bouquet of purple tulips a...,45394842176.png,"The image depicts a magnifying glass, commonly...",68016869942.png,"The image depicts a small, cute puppy standing...",77861539717.png,The image depicts an animated character with c...,97482048489.png,The image is a colorful and vibrant illustrati...,Let's break down the task step by step.\n\n## ...


In [4]:
image_columns = ["image1_name", "image2_name", "image3_name", "image4_name", "image5_name"]
caption_columns = ["image1_caption", "image2_caption", "image3_caption", "image4_caption", "image5_caption"]

def get_ordered_captions(row):
    image_caption_map = {row[img]: row[cap] for img, cap in zip(image_columns, caption_columns) if pd.notna(row[img])}
    ordered_captions = [image_caption_map.get(img, "") for img in eval(row["expected_order"]) if img in image_caption_map]
    return ordered_captions


df["ordered_captions"] = df.apply(get_ordered_captions, axis=1)


df_transformed = df[["compound", "sentence_type", "sentence", "expected_order", "ordered_captions"]]

In [18]:
df_transformed

,compound,sentence_type,sentence,expected_order,ordered_captions
0,elbow grease,idiomatic,It took a lot of elbow grease to get the old e...,"['35234427395.png', '53378381715.png', '399382...",[The image depicts a hand holding a sponge and...
1,night owl,idiomatic,"It's a constant battle for us, as he is a morn...","['61697797701.png', '93189810779.png', '893752...",[The image depicts a cartoon-style illustratio...
2,heart of gold,idiomatic,Even the somewhat seedy failed private eye has...,"['86137977215.png', '78062290185.png', '542405...",[The image depicts a cartoon scene where a you...
3,agony aunt,idiomatic,ESA's Space Weather Office is like Europe's st...,"['83600499282.png', '57658144685.png', '025128...","[The image depicts a person sitting at a desk,..."
4,shrinking violet,idiomatic,"This aged, rich wine is no shrinking violet wi...","['77861539717.png', '68016869942.png', '118443...",[The image depicts an animated character with ...
...,...,...,...,...,...
65,cold turkey,literal,"When you're serving cold turkey, take out only...","['47545447824.png', '78631376435.png', '162393...","[The image depicts a whimsical, cartoon-style ..."
66,low-hanging fruit,idiomatic,Howard said there was a lot of low-hanging fru...,"['87359560017.png', '28404294077.png', '911178...",[The image depicts a stylized target with thre...
67,busy bee,idiomatic,There's no need to worry if you're a busy bee ...,"['33246843335.png', '27532602036.png', '505168...",[The image depicts a person walking on a sidew...
68,wet blanket,literal,Wet blankets draped over neighboring cabins pr...,"['28147683669.png', '88635314726.png', '393854...",[The image depicts a rolled-up blanket with fu...


In [19]:
from langchain_community.llms import Ollama
import json

# Initialize the Ollama model
llm = Ollama(model="llama3.1")

def rank_captions_with_llm(llm, sentence, phrase, sentence_type, ordered_captions):
    """
    Uses LLM to rank captions based on relevance to the given sentence.
    """
    prompt = f"""
    You are an AI trained to rank image captions based on their relevance to a given sentence.

    ### **Inputs:**
    - **Sentence Type:** {sentence_type}
    - **Phrase:** {phrase}
    - **Sentence:** {sentence}
    - **Captions List:** {ordered_captions}

    ### **Task:**
    1. If the sentence type is **idiomatic**, understand the figurative meaning of the phrase.
    2. If the sentence type is **literal**, interpret the phrase in its direct, literal sense.
    3. Compare each caption in the list with the sentence and **rank them based on relevance** (1 being the most relevant).
    4. Return the rankings in **descending order of relevance**.

    ### **Output Format:**
    Provide the ranked captions in a structured JSON format:
    {{
        "ranked_captions": [
            {{"caption": "Most relevant caption", "rank": 1}},
            {{"caption": "Second most relevant caption", "rank": 2}},
            {{"caption": "Third most relevant caption", "rank": 3}},
            ...
        ]
    }}
    """

    response = llm.invoke(prompt)
    return response

def reverse_map_captions_to_images(row, ranked_captions):
    """
    Maps ranked captions back to their original image names in the expected order.
    """
    caption_to_image = {caption: img for caption, img in zip(row["ordered_captions"], eval(row["expected_order"]))}
    ranked_images = [caption_to_image.get(item["caption"], "") for item in ranked_captions]
    return ranked_images

def compare_image_orders(original_order, predicted_order):
    """
    Compares the predicted order of images with the original expected order.
    """
    matches = [1 if orig == pred else 0 for orig, pred in zip(original_order, predicted_order)]
    accuracy = sum(matches) / len(original_order)
    
    return {
        "original_order": original_order,
        "predicted_order": predicted_order,
        "accuracy": accuracy
    }

# Process the first row of the DataFrame
first_row = df_transformed.iloc[0]

# Rank captions using LLM
ranked_response = rank_captions_with_llm(llm, first_row["sentence"], first_row["compound"], first_row["sentence_type"], first_row["ordered_captions"])

# Convert LLM response to a structured format
ranked_captions = json.loads(ranked_response)["ranked_captions"]

# Reverse map captions to images
predicted_image_order = reverse_map_captions_to_images(first_row, ranked_captions)

# Compare predicted order with expected order
comparison_result = compare_image_orders(eval(first_row["expected_order"]), predicted_image_order)

# Display results
print(comparison_result)


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [21]:
print(ranked_response)

To solve this problem, we need to follow these steps:

### **Step 1: Understand the Figurative Meaning of the Phrase (Idiomatic)**
In this case, the phrase "elbow grease" is an idiom that means effort or hard work.

### **Step 2: Interpret the Sentence and Captions**

Given sentence: It took a lot of elbow grease to get the old engine running again.
This sentence implies that it required significant effort or hard work to repair the old engine.

### **Step 3: Compare Each Caption with the Sentence and Rank Them Based on Relevance**
Let's evaluate each caption in the list:

1. The first caption depicts someone cleaning a stove, which involves elbow grease (effort) but doesn't directly relate to fixing an engine.
2. The second caption shows someone dusting, which also requires effort but is unrelated to engine repair.
3. The third caption depicts someone painting a pipe, which again requires effort but has no connection to repairing an engine.
4. The fourth caption describes someone usin